In [ ]:
import matplotlib.pyplot as plt
import tifffile as tif
import pandas as pd
import PIL.Image as Image
import numpy as np
import colorsys
import zipfile
import os

def split_filepath(filepath):
    dirpath, filename = os.path.split(filepath)
    name, ext = os.path.splitext(filename)
    separator = '/' if (not dirpath or dirpath[-1] != '/') else ''
    return dirpath + separator, name, ext

def unzip_archive(root, filepath):
    dirpath, name, ext = split_filepath(filepath)
    if os.path.exists(root + dirpath + name):
        return False
    print("Unzipping archive: ", filepath)
    with zipfile.ZipFile(root + filepath, 'r') as zip_ref:
        zip_ref.extractall(root + dirpath)
    if os.path.exists(root + dirpath + name):
        return True
    print("!WARNING! Archive did not produce folder: ", root + filepath)
    return False

def enumerate_dataset(root, folder = '/'):
    print("Enumerating folder: ", folder)
    for filename in os.listdir(root + folder):
        filepath = folder + filename
        dirpath, name, ext = split_filepath(filepath)
        yield filepath, dirpath, name, ext

        if ext == '.zip':
            if unzip_archive(root, filepath):
                yield (dirpath + name), dirpath, name, ''
            else:
                continue
        elif ext:
            continue

        for fullpath, dirpath, name, ext in enumerate_dataset(root, dirpath + name + "/"):
            yield fullpath, dirpath, name, ext


In [ ]:
img_types = [".bmp", ".png", ".tif", ".tiff"]
misc_types = [".md", ".zip", ".txt", ".csv", ".py"]

files_by_type = {type:set() for type in (img_types + misc_types)}

dataroot = "../../data"
for filepath, dirpath, name, ext in enumerate_dataset(dataroot + "/raw", folder= "/zenodo/"):
    if not ext:
        continue
    files_by_type[ext].add(filepath)

# Display parallel histograms
plt.figure()
ax = plt.subplot(2, 1, 1)
plt.bar(img_types, [len(files_by_type[type]) for type in img_types], label='Files')
plt.title("File types in dataset")
plt.xlabel("File type")
plt.ylabel("Count")
plt.legend()
plt.show()


In [ ]:

LABELED = 'Labeled'
MASK = 'Mask'
UNLABELED = 'Unlabeled'
SYNTHETIC = 'Synthetic'

file_types = {cat:{type:set() for type in (img_types + misc_types)} for cat in [LABELED, MASK, UNLABELED, SYNTHETIC]}

def mask_map(category):
    if category == MASK:
        yield ("/Public/images", "/Public/labels")
        yield ("/Public/WSI", "/Public/WSI-labels")
        yield ("/Training-labeled/images", "/Training-labeled/labels")
        yield ("/Tuning/images", "/Tuning/labels")
    if category == SYNTHETIC:
        yield ("/Hidden/images", "/Hidden/osilab_seg")
        yield ("/Public/images", "/Public/1st_osilab_seg")
        yield ("/Public/WSI", "/Public/osilab_seg_WSI")

def categorize(dirpath):
    for (img, mask) in mask_map(MASK):
        if mask in dirpath:
            return MASK
        elif img in dirpath:
            return LABELED
    for (img, mask) in mask_map(SYNTHETIC):
        if mask in dirpath:
            return SYNTHETIC
    return UNLABELED

for type, paths in files_by_type.items():
    for filepath in paths:
        file_types[categorize(filepath)][type].add(filepath)

In [ ]:
for cat in [LABELED, MASK, UNLABELED, SYNTHETIC]:
    types = file_types[cat]
    counts = {k:len(s) for k, s in types.items()}
    print(cat, sum(counts.values()), counts)

x = np.arange(len(img_types))  # the label locations
width = 0.25

# Display parallel histograms
plt.figure()
ax = plt.subplot(2, 1, 1)

index = 0
for cat, data in file_types.items():
    plt.bar(x + index * width, [len(data[key]) for key in img_types], width, label=cat)
    index += 1
    
plt.title("File types in dataset")
plt.xlabel("File type")
plt.xticks(x + width, img_types)
plt.ylabel("Count")
plt.legend()
plt.show()

In [ ]:
mask_assoc = {}
visited = set(file_types[MASK][".tiff"])

def get_mask_path(img_path):
    folder, name, ext = split_filepath(img_path)
    if "WSI" in folder:
        return folder.replace("/WSI", "/WSI-labels") + name + "_label.tiff"
    if "/images" in folder:
        return folder.replace("/images", "/labels") + name + "_label.tiff"
    return None

for ext in img_types:
    for path in file_types[LABELED][ext]:
        mask_path = get_mask_path(path)
        if mask_path and os.path.exists(dataroot + "/raw" + mask_path):
            mask_assoc[path] = mask_path
            visited.remove(mask_path)
        else:
            print("Missing mask: ", dataroot + "/raw", mask_path, path)

print(len(mask_assoc))
print(visited)
assert not visited

In [ ]:
synth_assoc = {}
visited = set(file_types[SYNTHETIC][".tiff"])

def get_synth_path(img_path):
    folder, name, ext = split_filepath(img_path)
    if "/Hidden" in folder:
        return folder.replace("/images", "/osilab_seg") + name + "_label.tiff"
    elif "/Public/images" in folder:
        return folder.replace("/images", "/1st_osilab_seg") + name + "_label.tiff"
    elif "/Public/WSI" in folder:
        return folder.replace("/WSI", "/osilab_seg_WSI") + name + "_label.tiff"
    return None

for cat in [UNLABELED, LABELED]:
    for ext in img_types:
        for path in file_types[cat][ext]:
            synth_path = get_synth_path(path)
            if synth_path and os.path.exists(dataroot + "/raw" + synth_path):
                synth_assoc[path] = synth_path
                visited.remove(synth_path)

print(len(synth_assoc))
print(visited)
assert not visited

In [126]:
def dataset_frame(root, assoc):
    expected = len(assoc)

    numbers = []
    for img_path, datapath in assoc.items():
        print(len(numbers), "/", expected)
        # Get global statistics
        imgT = tif.imread(root + datapath)
        numbers.append({"Path":img_path, "Mask":datapath, "Width": imgT.shape[1], "Height":imgT.shape[0], "Objects": imgT.max(), "Background": (imgT == 0).sum()})

    return pd.DataFrame(numbers, columns = ["Path", "Mask", "Width", "Height", "Objects", "Background"]).set_index("Path")

def save_maskframes(root, assoc, store):
    for index, (img_path, mask_path) in enumerate(assoc.items()):
        print(index, "/", len(assoc))
        folder, name, ext = split_filepath(mask_path)
        maskframe = mask_frame(root, mask_path)
        target = store + folder + name
        maskframe.to_csv(target + ".csv")

def merge_lists(compare, merge, listA, listB):
    merged = [0] * (len(listA) + len(listB))
    indexM, indexA, indexB = 0, 0, 0
    while indexA < len(listA) and indexB < len(listB):
        cmp = compare(listA[indexA], listB[indexB])
        if cmp < 0:
            merged[indexM] = listA[indexA]
            indexM += 1
            indexA += 1
        elif cmp > 0:
            merged[indexM] = listB[indexB]
            indexM += 1
            indexB += 1
        else:
            merged[indexM] = merge(listA[indexA], listB[indexB])
            indexM += 1
            indexA += 1
            indexB += 1
            merged.pop()
    if indexA < len(listA):
        merged[indexM:] = listA[indexA:]
    if indexB < len(listB):
        merged[indexM:] = listB[indexB:]
    return merged

def mask_frame_leaf(tensor, bounds):
    tensor = tensor[bounds.top:bounds.bottom, bounds.left:bounds.right]
    objectIDs = np.unique(tensor)
    return [detectObject(tensor, bounds, id) for id in objectIDs]

def mask_frame(root, mask_path):
    tensor = tif.imread(root + mask_path)
    bounds = BoundingBox()
    bounds.right = tensor.shape[1]
    bounds.bottom = tensor.shape[0]
    print(bounds.width(), "x", bounds.height())
    print(tensor.max(), "objects")
    objects = mask_frame_branch(tensor, bounds)
    for object in objects:
        normalizeObject(object)
    return pd.DataFrame(objects, columns = ["ID", "X", "Y", "Left", "Right", "Top", "Bottom", "Area"]).set_index("ID")

def mask_frame_branch(tensor, bounds):
    if bounds.small():
        return mask_frame_leaf(tensor, bounds)
    splitA, splitB = bounds.split()
    objectsA, objectsB = mask_frame_branch(tensor, splitA), mask_frame_branch(tensor, splitB)
    merged = merge_lists(compareObjects, mergeObjects, objectsA, objectsB)
    return merged

class BoundingBox:
    def __init__(self):
        self.left = 0
        self.right = 0
        self.top = 0
        self.bottom = 0
    
    def copy(self):
        bounds = BoundingBox()
        bounds.left = self.left
        bounds.right = self.right
        bounds.top = self.top
        bounds.bottom = self.bottom
        return bounds
    
    def width(self):
        return self.right - self.left
    
    def height(self):
        return self.bottom - self.top
    
    def small(self):
        return self.width() < 128 and self.height() < 128
    
    def split(self):
        boundsA = self.copy()
        boundsB = self.copy()
        if self.height() < self.width():
            #split = self.left + int(np.exp2(np.ceil(np.log2(self.width()) - 1)))
            split = self.left + self.width() // 2
            boundsA.right = split
            boundsB.left = split
        else:
            #split = self.top + int(np.exp2(np.ceil(np.log2(self.height()) - 1)))
            split = self.top + self.height() // 2
            boundsA.bottom = split
            boundsB.top = split
        return (boundsA, boundsB)

def normalizeObject(object):
    object["X"] = int(np.round(object["X"]))
    object["Y"] = int(np.round(object["Y"]))
    return object

def detectObject(tensor, bounds, id):
    # Get local statistics
    rows, cols = np.where(tensor == id)
    x = bounds.left + cols.mean()
    y = bounds.top + rows.mean()
    left = bounds.left + cols.min()
    right = bounds.left + cols.max() + 1
    top = bounds.top + rows.min()
    bottom = bounds.top + rows.max() + 1
    area = len(rows)
    return {"ID": id, "X": x, "Y": y, "Left": left, "Right": right, "Top": top, "Bottom": bottom, "Area": area}

def compareObjects(objectA, objectB):
    return objectA["ID"] - objectB["ID"]

def mergeObjects(objectA, objectB):
    assert objectA["ID"] == objectB["ID"]
    area = objectA["Area"] + objectB["Area"]
    x = (objectA["X"] * objectA["Area"] + objectB["X"] * objectB["Area"]) / area
    y = (objectA["Y"] * objectA["Area"] + objectB["Y"] * objectB["Area"]) / area
    left = min(objectA["Left"], objectB["Left"])
    right = max(objectA["Right"], objectB["Right"])
    top = min(objectA["Top"], objectB["Top"])
    bottom = max(objectA["Bottom"], objectB["Bottom"])
    return {"ID": objectA["ID"], "X": x, "Y": y, "Left":left, "Right":right, "Top": top, "Bottom":bottom, "Area": area}


In [ ]:
data_map = dataset_frame(dataroot + "/raw", mask_assoc)
data_map.to_csv(dataroot + "/zenodo.labels.csv")

synth_map = dataset_frame(dataroot + "/raw", synth_assoc)
synth_map.to_csv(dataroot + "/zenodo.synth.csv")

save_maskframes(dataroot + "/raw", mask_assoc, dataroot + "/processed")
save_maskframes(dataroot + "/raw", synth_assoc, dataroot + "/processed")
#df = mask_frame(dataroot + "/raw", "/zenodo/Testing/Public/WSI-labels/Tonsil_label.tiff")

In [ ]:

# Save black-white mask
def save_bw_mask(root, store, datapath):
    imgT = tif.imread(root + datapath)
    im = Image.fromarray((imgT != 0).astype('uint8')*255)
    folder, name, ext = split_filepath(datapath)
    target = store + folder
    os.makedirs(target, exist_ok=True)
    maskfile = target + name + ".png"
    im.save(maskfile)

# TOO SLOW
# Save hue-vector mask
def save_hue_mask(root, store, datapath):
    imgT = tif.imread(root + datapath)
    imgTC = np.zeros((imgT.shape[0], imgT.shape[1], 3), dtype=np.float32)
    N = imgT.max()
    for i in range(1, N+1):
        rows, cols = np.where(imgT == i)
        bounds = [rows.min(), rows.max(), cols.min(), cols.max()]
        middle = np.array([rows.mean(), cols.mean()])
        for r, c in zip(rows, cols):
            location = np.array([r, c])
            vector = (location - middle)/np.array([bounds[1] - bounds[0], bounds[3] - bounds[2]])
            # Get euclidian norm
            norm = np.linalg.norm(vector)
            angle = np.arctan2(vector[1], vector[0])  # Calculate the angle in radians
            hue = (angle + np.pi) / (2 * np.pi)  # Normalize angle to [0, 1] for hue
            rgb = colorsys.hsv_to_rgb(hue, norm, 1.0)  # Convert HSV to RGB
            imgTC[r, c] = rgb
    print(imgTC.min(), imgTC.max())
    im = Image.fromarray((imgTC * 255).astype('uint8'), mode="RGB")
    folder, name, ext = split_filepath(datapath)
    target = store + folder
    os.makedirs(target, exist_ok=True)
    maskfile = target + name + ".vect.png"
    im.save(maskfile)

# Save vector mask (tiff)
def save_vector_mask(root, framename, datapath):
    imgT = tif.imread(root + datapath)
    imgTC = np.zeros((imgT.shape[0], imgT.shape[1], 3), dtype=np.float32)
    N = imgT.max()
    for i in range(1, N+1):
        rows, cols = np.where(imgT == i)
        bounds = [rows.min(), rows.max(), cols.min(), cols.max()]
        middle = np.array([rows.mean(), cols.mean()])
        for r, c in zip(rows, cols):
            location = np.array([r, c])
            vector = (location - middle)/np.array([bounds[1] - bounds[0], bounds[3] - bounds[2]])
            # Get euclidian norm
            norm = np.linalg.norm(vector)
            angle = np.arctan2(vector[1], vector[0])  # Calculate the angle in radians
            hue = (angle + np.pi) / (2 * np.pi)  # Normalize angle to [0, 1] for hue
            rgb = colorsys.hsv_to_rgb(hue, norm, 1.0)  # Convert HSV to RGB
            imgTC[r, c] = rgb
    print(imgTC.min(), imgTC.max())
    im = Image.fromarray((imgTC * 255).astype('uint8'), mode="RGB")
    folder, name, ext = split_filepath(datapath)
    target = root + framename + folder
    maskfile = target + name + ".vect.png"
    im.save(maskfile)


In [ ]:
def enumerate_datasets(dataroot):
    for filename in os.listdir(dataroot):
        dirpath, name, ext = split_filepath(filename)
        if ext == ".csv":
            yield pd.read_csv(dataroot + dirpath + filename)

def preprocess_masks(dataroot, color = False):
    rawroot = dataroot + "/raw"
    procroot = dataroot + "/processed"
    for df in enumerate_datasets(dataroot):
        for index, datapath in enumerate(df["Mask"]):
            print(index, "/", len(df))
            if color:
                save_hue_mask(rawroot, procroot, datapath)
            else:
                save_bw_mask(rawroot, procroot, datapath)

preprocess_masks(dataroot)
#save_hue_mask(dataroot + "/raw", dataroot + "/processed", "/zenodo/Testing/Public/labels/OpenTest_041_label.tiff")

In [ ]:
#data_map["Objects"] < 2000
data_map

In [ ]:
print("Valeurs aberrantes?")
data_map[data_map["Objects"] > 2000]

In [ ]:

num_obj = data_map["Objects"]
print(num_obj[num_obj > 2000])
num_obj = num_obj[num_obj < 2000]
print(len(num_obj), sum(num_obj), set(num_obj))

# Display parallel histograms
plt.figure()
plt.subplot(2, 1, 1)
plt.hist(np.log2([x for x in num_obj]), bins=100)
plt.title("Number of segmented objects per file")
plt.xlabel("Number of objects (log2)")
#plt.xticks([2.0 ** x for x in range(5)])
plt.ylabel("Count")
plt.show()

In [ ]:
print(len(set(n for n in (data_map["Width"] * data_map["Height"]))))

area = data_map["Width"] * data_map["Height"]
print("Too big:", area[area > 10 ** 7].count())
#area = area[area < 10 ** 7]

# Display parallel histograms
plt.figure()
plt.subplot(2, 1, 1)
plt.hist(np.log2(area), bins=100)
#plt.hist([np.log2(n) for n in numbers if n < 2000], bins=100)
plt.title("Size of of mask files")
plt.xlabel("Size of file (log2)")
#plt.xticks([2.0 ** x for x in range(5)])
plt.ylabel("Count")
plt.show()

In [ ]:
#print(set(n[2] for n in numbers))

density = 1 - (data_map["Background"] / data_map["Width"] / data_map["Height"])

plt.figure()
plt.subplot(2, 1, 1)
plt.hist(density, bins=100)
#plt.hist([-np.log2(n[2]) for n in numbers], bins=100)
plt.title("Density of segmented objects per file")
plt.xlabel("Density of objects")
#plt.xticks([2.0 ** x for x in range(5)])
plt.ylabel("Count")
plt.show()

In [ ]:
radius = (density / data_map["Objects"] / np.pi) ** .5

plt.figure()
plt.subplot(2, 1, 1)
plt.hist(radius, bins=100)
#plt.hist([-np.log2(n[2]) for n in numbers], bins=100)
plt.title("Average radius of segmented objects per file")
plt.xlabel("Radius of objects (in %)")
#plt.xticks([2.0 ** x for x in range(8)])
plt.ylabel("Count")
plt.show()